In [351]:
from db_utils import fetch_data_from_db
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
import sklearn.metrics
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from imblearn.combine import SMOTEENN
import pickle

In [352]:
sql_query = "SELECT * FROM prod_Churn;"

data = fetch_data_from_db(sql_query)

df = pd.DataFrame(data)

df.head()

,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,Internet_Service,...,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Customer_Status,Tenure_Group,Age_Group
0,1.0,-0.731336,0.0,4.0,-0.095158,0.906857,5.0,1.0,0.0,1.0,...,1.0,0.016494,-0.810520,-0.252777,-0.276148,-0.486405,-0.790785,0.0,3.0,3.0
1,0.0,-0.136230,1.0,12.0,1.419340,-0.414891,5.0,1.0,1.0,1.0,...,1.0,-2.223950,-0.832970,4.499950,0.117497,-0.820172,-0.918424,0.0,1.0,5.0
2,1.0,0.220834,0.0,21.0,-0.744230,1.662140,4.0,1.0,0.0,1.0,...,0.0,0.283674,-0.948328,-0.252777,-0.276148,-0.775261,-0.986643,1.0,4.0,6.0
3,1.0,1.887130,0.0,9.0,-0.960587,0.340393,3.0,1.0,0.0,1.0,...,0.0,1.059460,-0.526238,-0.252777,-0.276148,-0.509628,-0.572048,1.0,2.0,11.0
4,0.0,1.946640,0.0,17.0,-0.960587,-0.886944,5.0,1.0,0.0,1.0,...,1.0,0.605578,-0.954260,-0.252777,-0.276148,-0.906826,-1.030730,1.0,5.0,11.0


In [353]:
x=df.drop('Customer_Status',axis=1)
x

,Gender,Age,Married,State,Number_of_Referrals,Tenure_in_Months,Value_Deal,Phone_Service,Multiple_Lines,Internet_Service,...,Paperless_Billing,Payment_Method,Monthly_Charge,Total_Charges,Total_Refunds,Total_Extra_Data_Charges,Total_Long_Distance_Charges,Total_Revenue,Tenure_Group,Age_Group
0,1.0,-0.731336,0.0,4.0,-0.095158,0.906857,5.0,1.0,0.0,1.0,...,1.0,1.0,0.016494,-0.810520,-0.252777,-0.276148,-0.486405,-0.790785,3.0,3.0
1,0.0,-0.136230,1.0,12.0,1.419340,-0.414891,5.0,1.0,1.0,1.0,...,0.0,1.0,-2.223950,-0.832970,4.499950,0.117497,-0.820172,-0.918424,1.0,5.0
2,1.0,0.220834,0.0,21.0,-0.744230,1.662140,4.0,1.0,0.0,1.0,...,1.0,0.0,0.283674,-0.948328,-0.252777,-0.276148,-0.775261,-0.986643,4.0,6.0
3,1.0,1.887130,0.0,9.0,-0.960587,0.340393,3.0,1.0,0.0,1.0,...,1.0,0.0,1.059460,-0.526238,-0.252777,-0.276148,-0.509628,-0.572048,2.0,11.0
4,0.0,1.946640,0.0,17.0,-0.960587,-0.886944,5.0,1.0,0.0,1.0,...,1.0,1.0,0.605578,-0.954260,-0.252777,-0.276148,-0.906826,-1.030730,5.0,11.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6002,0.0,-0.374272,0.0,19.0,-0.311516,1.473320,5.0,1.0,1.0,1.0,...,1.0,1.0,0.299769,-0.676682,-0.252777,-0.276148,-0.888318,-0.804825,4.0,4.0
6003,1.0,0.220834,1.0,6.0,-1.609660,0.151572,0.0,0.0,0.0,1.0,...,0.0,1.0,-0.638579,0.312626,-0.252777,-0.276148,-0.932727,-0.032720,2.0,6.0
6004,0.0,1.053980,1.0,10.0,1.419340,0.718036,5.0,1.0,1.0,1.0,...,1.0,0.0,0.317474,0.193563,-0.252777,-0.276148,-0.231219,0.082720,3.0,8.0
6005,1.0,-1.445460,0.0,13.0,-1.176940,-0.698123,5.0,1.0,1.0,1.0,...,1.0,1.0,0.162960,0.904036,-0.252777,-0.276148,1.532820,1.174730,5.0,1.0


In [354]:
y=df['Customer_Status']
y

0       0.0
1       0.0
2       1.0
3       1.0
4       1.0
       ... 
6002    1.0
6003    0.0
6004    1.0
6005    0.0
6006    0.0
Name: Customer_Status, Length: 6007, dtype: float64

In [355]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [356]:
print(y_train.value_counts())

Customer_Status
0.0    3395
1.0    1410
Name: count, dtype: int64


In [357]:
sm = SMOTEENN()
X_resampled, y_resampled = sm.fit_resample(x,y)

In [358]:
x_train_smoteen,x_test_smoteen,y_train_smoteen,y_test_smoteen=train_test_split(X_resampled, y_resampled,test_size=0.2)


In [359]:
print(y_train_smoteen.value_counts())

Customer_Status
1.0    2860
0.0    1646
Name: count, dtype: int64


In [360]:
# dictionary of models
models = {
    "Decision Tree": DecisionTreeClassifier(criterion = "gini",random_state = 100,max_depth=6, min_samples_leaf=8),
    "Random Forest": RandomForestClassifier(n_estimators=100, criterion='gini', random_state = 100,max_depth=6, min_samples_leaf=8),
    "XGBoost":XGBClassifier(eval_metric='mlogloss', random_state=100)
}

In [361]:
# dictionary to store the cross validation results
cv_scores = {}

# perform 5-fold cross validation for each model
for model_name, model in models.items():
  print(f"Training {model_name} with default parameters")
  scores = cross_val_score(model, x_train_smoteen, y_train_smoteen, cv=5, scoring="accuracy")
  cv_scores[model_name] = scores
  accuracy_score = np.mean(scores)*100
  print(f"{model_name} cross-validation accuracy: {accuracy_score:.3f}%")
  print("-"*70)

Training Decision Tree with default parameters
Decision Tree cross-validation accuracy: 94.541%
----------------------------------------------------------------------
Training Random Forest with default parameters
Random Forest cross-validation accuracy: 94.541%
----------------------------------------------------------------------
Training XGBoost with default parameters
XGBoost cross-validation accuracy: 96.316%
----------------------------------------------------------------------


In [362]:
cv_scores

{'Decision Tree': array([0.94456763, 0.93562708, 0.94672586, 0.95116537, 0.94894562]),
 'Random Forest': array([0.94345898, 0.9445061 , 0.95560488, 0.9445061 , 0.93895671]),
 'XGBoost': array([0.96452328, 0.95893452, 0.96892342, 0.96115427, 0.96226415])}

### Model Selection Insight:

We are moving forward with AdaBoost as it provides the best performance on our current dataset with an accuracy of 96.83%. Since the dataset is small, AdaBoost offers a good balance of performance and efficiency. If the dataset were larger, XGBoost might be a better choice due to its ability to handle larger datasets more efficiently.

In [363]:
xgboost_model = XGBClassifier(eval_metric='mlogloss', random_state=100)

In [364]:
xgboost_model.fit(x_train_smoteen, y_train_smoteen)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=100, ...)

### Model Evaluation

In [365]:
y_test_pred = xgboost_model.predict(x_test)

print("Accuracy Score:\n", sklearn.metrics.accuracy_score(y_test, y_test_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_test_pred))
print("Classification Report:\n", classification_report(y_test, y_test_pred))

Accuracy Score:
 0.7945091514143094
Confusion Matrix:
 [[674 206]
 [ 41 281]]
Classification Report:
               precision    recall  f1-score   support

         0.0       0.94      0.77      0.85       880
         1.0       0.58      0.87      0.69       322

    accuracy                           0.79      1202
   macro avg       0.76      0.82      0.77      1202
weighted avg       0.84      0.79      0.80      1202



### Model Saving

In [366]:
model_data = {"model": xgboost_model, "features_names": x.columns.tolist()}

with open("customer_churn_model.pkl", "wb") as f:
  pickle.dump(model_data, f)

loaded_model = model_data["model"]
feature_names = model_data["features_names"]

### Load the saved  model and  build a Predictive System

with open("customer_churn_model.pkl", "rb") as f:
  model_data = pickle.load(f)

loaded_model = model_data["model"]
feature_names = model_data["features_names"]

In [367]:
print(loaded_model)
print(feature_names)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=100, ...)
['Gender', 'Age', 'Married', 'State', 'Number_of_Referrals', 'Tenure_in_Months', 'Value_Deal', 'Phone_Service', 'Multiple_Lines', 'Internet_Service', 'Internet_Type', 'Online_Security', 'Online_Backup', 'Device_Protection_Plan', 'Premium

In [368]:
sql_query = "SELECT * FROM new_customers;"
data = fetch_data_from_db(sql_query)

input_data_df = pd.DataFrame(data)

In [369]:
bins = list(range(1, 37, 6))  
bins.append(37)
labels = ["{0} - {1}".format(bins[i], bins[i + 1] - 1) for i in range(len(bins) - 1)]
input_data_df['Tenure_Group'] = pd.cut(input_data_df['Tenure_in_Months'], bins=bins, right=False, labels=labels)

bins = list(range(1, 85, 6))
bins.append(85) 
labels = ["{0} - {1}".format(bins[i], bins[i + 1] - 1) for i in range(len(bins) - 1)]
input_data_df['Age_Group'] = pd.cut(input_data_df['Age'], bins=bins, right=False, labels=labels)

In [370]:
input_data_df = input_data_df.dropna(how='any')

In [371]:
output_data_df = input_data_df.copy()

In [372]:
input_data_df = input_data_df.drop(columns=["Customer_Status", "Customer_ID", "id", "Churn_Category", "Churn_Reason"])

In [373]:
with open("encoders.pkl", "rb") as f:
    encoders = pickle.load(f)

with open("standardScalers.pkl", "rb") as f:
    scalers = pickle.load(f)

for column, transformer in encoders.items():
    if column in input_data_df.columns:
        input_data_df[column] = transformer.transform(input_data_df[column])

for column, scaler in scalers.items():
    if column in input_data_df.columns:
        input_data_df[column] = scaler.transform(input_data_df[[column]])

In [374]:
predictions = loaded_model.predict(input_data_df)
pred_probs = loaded_model.predict_proba(input_data_df)

In [375]:
output_data_df['Prediction'] = ['Churn' if pred == 1 else 'No Churn' for pred in predictions]
output_data_df['Prediction_Probability'] = [prob[1] for prob in pred_probs]

In [377]:
output_file = "prediction_results.csv"
output_data_df.to_csv(output_file, index=False)

print(f"Prediction results have been saved to {output_file}")

Prediction results have been saved to prediction_results.csv
